In [1]:
from htmd import *
htmd.config(viewer='webgl')
#os.chdir('/webdata/73hboiwia98hdj209jq0/')  # Skip this command.


Please cite -- HTMD: High-Throughput Molecular Dynamics for Molecular Discovery
J. Chem. Theory Comput., 2016, 12 (4), pp 1845-1852. 
http://pubs.acs.org/doi/abs/10.1021/acs.jctc.6b00049

You are on the latest HTMD version (1.5.3).



# System Building μ-opioid Receptor in Membrane
by [Stefan Doerr](https://github.com/stefdoerr)
&nbsp;

**Download all the required files for the tutorial from this** [link](http://pub.htmd.org/73hboiwia98hdj209jq0/mor.tar.gz)

You can watch the presentation here:

[![](http://pub.htmd.org/73hboiwia98hdj209jq0/opioid_youtube.png)](https://youtu.be/DF9cHKBX19A?t=22m17s)

## Prepare the protein

In [2]:
prot = Molecule('mor/4dkl.pdb')
prot.filter('protein and noh and chain B or water within 5 of (chain B and protein)')

2016-11-09 11:05:55,705 - htmd.molecule.molecule - INFO - Removed 5120 atoms. 2262 atoms remaining in the molecule.


Automatically detecting segments and assigning names to them.

In [3]:
prot = autoSegment(prot,'protein','P')

2016-11-09 11:05:58,359 - htmd.builder.builder - INFO - Created segment P0 between resid 65 and 263.
2016-11-09 11:05:58,360 - htmd.builder.builder - INFO - Created segment P1 between resid 270 and 352.


## Building the protein

In [4]:
topos  = ['mor/ff.rtf','top/top_all36_prot.rtf','top/top_all36_lipid.rtf','top/top_water_ions.rtf']
params = ['mor/ff.prm','par/par_all36_prot.prm','par/par_all36_lipid.prm','par/par_water_ions.prm']

prot = charmm.build(prot, topo=topos, param=params, 
                    outdir='./03c_morbuild/prot/', ionize=False)

2016-11-09 11:06:02,167 - htmd.builder.charmm - INFO - Writing out segments.


Bond between A: [serial 3005 resid 140 resname CYS chain B segid P0]
             B: [serial 3615 resid 217 resname CYS chain B segid P0]



2016-11-09 11:06:03,126 - htmd.builder.builder - INFO - One disulfide bond was added
2016-11-09 11:06:03,267 - htmd.builder.charmm - INFO - Starting the build.
2016-11-09 11:06:03,340 - htmd.builder.charmm - WARNING - Failed to guess coordinates for 1 atoms due to bad angles.
2016-11-09 11:06:03,341 - htmd.builder.charmm - WARNING - Poorly guessed coordinates for 78 atoms.
2016-11-09 11:06:03,341 - htmd.builder.charmm - WARNING - Please check /home/ec2-user/03_PreparationBuilding_TGiorgino/morbuild/prot/log.txt for further information.
2016-11-09 11:06:03,342 - htmd.builder.charmm - INFO - Finished building.


In [5]:
prot.reps.add(sel='segid P0', style='NewCartoon', color=1)
prot.reps.add(sel='segid P1', style='NewCartoon', color=2)
prot.view()

## Add a sodium in the receptor

In [6]:
sod = Molecule('mor/sod.pdb')
sod.set('segid','S1')
prot.append(sod)
prot.reps.add(sel='ions', style='VDW')
prot.view()

## Embed the protein into a membrane

In [7]:
memb = Molecule('mor/membrane80by80C36.pdb')

Center the membrane onto the protein center

In [8]:
pcenter = np.mean(prot.get('coords','protein'),axis=0)
mcenter = np.mean(memb.get('coords'),axis=0)
memb.moveBy(pcenter-mcenter)

And now embed

In [9]:
mol = embed(prot,memb)

## Visualize the embedded system

In [10]:
mol.reps.add(sel='protein', style='NewCartoon', color='Secondary Structure')
mol.reps.add(sel='ions', style='VDW')
mol.reps.add(sel='lipids', style='Lines')
mol.view()

## Add a ligand

In [11]:
lig = Molecule('mor/QM-min.pdb') 
lig.set('segid','L');
lcenter = np.mean(lig.get('coords'),axis=0)
newlcenter=[random.uniform(-10, 10), random.uniform(-10, 10),  43 ]
lig.rotateBy(uniformRandomRotation(), lcenter)
lig.moveBy(newlcenter-lcenter)
mol.append(lig)

## Put it in a water box

In [12]:
coo = mol.get('coords','noh and (lipids or protein)')
m = np.min(coo, axis=0) + [0, 0, -5]
M = np.max(coo, axis=0) + [0, 0, 20]
smol = solvate(mol, minmax=np.vstack((m,M)))

2016-11-09 11:06:46,224 - htmd.builder.solvate - INFO - Using water pdb file at: /home/ec2-user/miniconda3/lib/python3.5/site-packages/htmd/builder/wat.pdb
2016-11-09 11:06:47,287 - htmd.builder.solvate - INFO - Replicating 8 water segments, 2 by 2 by 2


Solvating: 100% (8/8) [############################################] eta 00:00 /


2016-11-09 11:07:15,194 - htmd.builder.solvate - INFO - After removing water molecules colliding with other molecules, 9118 water molecules were added to the system.


## Visualize

In [13]:
smol.reps.add(sel='segid L', style='Licorice')
smol.reps.add(sel='water', style='Lines')
smol.view()

## Build!

In [14]:
molbuilt = charmm.build(smol, topo=topos, param=params, 
                        outdir='./03c_morbuild/build/', saltconc=0.15)

2016-11-09 11:08:14,722 - htmd.builder.charmm - INFO - Writing out segments.


Bond between A: [serial 23211 resid 140 resname CYS chain  segid P0]
             B: [serial 24447 resid 217 resname CYS chain  segid P0]



2016-11-09 11:09:45,748 - htmd.builder.builder - INFO - One disulfide bond was added
2016-11-09 11:09:45,886 - htmd.builder.charmm - INFO - Starting the build.
2016-11-09 11:09:46,455 - htmd.builder.charmm - INFO - Finished building.
2016-11-09 11:09:49,697 - htmd.builder.ionize - INFO - Adding 14 anions + 0 cations for neutralizing and 66 ions for the given salt concentration.
2016-11-09 11:09:50,131 - htmd.builder.ionize - INFO - Min distance of ions from molecule: 5A
2016-11-09 11:09:50,132 - htmd.builder.ionize - INFO - Min distance between ions: 5A
2016-11-09 11:09:50,132 - htmd.builder.ionize - INFO - Placing 80 ions.
2016-11-09 11:10:33,825 - htmd.builder.charmm - INFO - Writing out segments.
2016-11-09 11:12:05,869 - htmd.builder.charmm - INFO - Starting the build.
2016-11-09 11:12:06,446 - htmd.builder.charmm - INFO - Finished building.


## Visualize built system

In [15]:
molbuilt.view(sel='protein', style='NewCartoon', color='Secondary Structure', hold=True)
molbuilt.view(sel='segid L', style='Licorice', hold=True)
molbuilt.view(sel='lipids', style='Lines', hold=True)
molbuilt.view(sel='ions', style='VDW', hold=True)
molbuilt.view(sel='water', style='Lines')